In [87]:
import pandas as pd
import numpy as np
from utils.metrics import NRMSE
import os


def get_tfe(model_results):
    baseline_result  = model_results[model_results.eblc == 'baseline']
    model_results['TFE'] = (model_results.nrmse.values - baseline_result.nrmse.values)/baseline_result.nrmse.values
    baseline_result = baseline_result.append([baseline_result]*2, ignore_index=True)
    baseline_result.at[0, 'eblc'] = 'pmc'
    baseline_result.at[1, 'eblc'] = 'sz'
    baseline_result.at[2, 'eblc'] = 'swing'
    baseline_result['TFE'] = 0.
    return pd.concat([model_results, baseline_result])


def join_with_aus():
    aus_results = pd.DataFrame()
    for root, dirs, files in os.walk('../results/tfe/per_model/'):
        for result_file in files:
            # if 'gru' in result_file or 'arima' in result_file:
            #     continue
            print(result_file)
            model_results = pd.read_csv(os.path.join(root, result_file))
            model_results = model_results[(model_results.data == 'aus')]
            model_results['model'] = result_file.split('_')[0]
            model_results = get_tfe(model_results)
            aus_results = pd.concat([aus_results, model_results])


    aus_results.reset_index(drop=True, inplace=True)
    aus_results.eblc = aus_results.eblc.str.upper()
    aus_results.model = aus_results.model.str.upper()
    aus_results.data = aus_results.data.str.upper()
    aus_results['tfe_metric'] = 'nrmse'
    aus_results['te_metric'] = 'nrmse'
    aus_results['te'] = aus_results['error']
    x = aus_results.eb.values
    aus_results.eb = np.where(x >= 1.0, x*0.01, x)
    aus_results = aus_results.rename({'nrmse': 'forecasting error', 'eblc': 'compression'}, axis=1)
    aus_results = aus_results.drop(['mae', 'rmse', 'rse', 'corr', 'data_corr'], axis=1)
    all_model_results = pd.read_csv('../results/tfe/all_models_results.csv')
    all_model_results = all_model_results[all_model_results.data != 'AUS']
    cr_df = pd.read_csv('../results/cr/all_cr.csv')
    cr_df = cr_df[(cr_df.data == 'aus')]
    cr_df.compression = cr_df.compression.str.upper()
    cr_df.data = cr_df.data.str.upper()
    cr_df.rename({'gzip': 'compression ratio', 'error_bound': 'eb'}, axis=1, inplace=True)
    cr_df.set_index(['data', 'compression', 'eb'], inplace=True)
    joined = cr_df.merge(aus_results, on=['data', 'compression', 'eb'])
    namr = pd.concat([all_model_results, joined]).reset_index(drop=True)
    namr.to_csv('../results/tfe/all_models_results_.csv', index=False)


In [88]:
join_with_aus()

arima_results.csv
dlinear_results.csv
gru_results.csv
informer_results.csv
nbeats_results.csv
transformer_results.csv


In [89]:
def join_with_arima():
    df = pd.read_csv('../results/tfe/per_model/arima_results.csv')
    df = df[['error', 'nrmse', 'eblc', 'data', 'eb']]
    arima_results = pd.DataFrame()
    for data in df.data.unique():
        data_results = df[df.data == data]
        get_tfe(data_results)
        arima_results = pd.concat([arima_results, data_results])
    arima_results.reset_index(drop=True, inplace=True)
    arima_results.eblc = arima_results.eblc.str.upper()
    arima_results['model'] = 'ARIMA'
    arima_results.data = arima_results.data.str.upper()
    arima_results['tfe_metric'] = 'nrmse'
    arima_results['te_metric'] = 'nrmse'
    arima_results['te'] = arima_results['error']
    x = arima_results.eb.values
    arima_results.eb = np.where(x >= 1.0, x*0.01, x)
    arima_results = arima_results.rename({'nrmse': 'forecasting error', 'eblc': 'compression'}, axis=1)
    all_model_results = pd.read_csv('../results/tfe/all_models_results_.csv')
    cr_df = pd.read_csv('../results/cr/all_cr.csv')
    cr_df.compression = cr_df.compression.str.upper()
    cr_df.data = cr_df.data.str.upper()
    cr_df.rename({'gzip': 'compression ratio', 'error_bound': 'eb'}, axis=1, inplace=True)
    cr_df.set_index(['data', 'compression', 'eb'], inplace=True)
    joined = cr_df.merge(arima_results, on=['data', 'compression', 'eb'])
    namr = pd.concat([all_model_results, joined]).reset_index(drop=True)
    namr.to_csv('../results/tfe/all_models_results_.csv', index=False)

In [90]:
join_with_arima()

D:\Anaconda\envs\dart\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
def join_arima_pweather():
    df = pd.read_csv('../results/tfe/per_model/arima_results.csv')
    df = df[['error', 'nrmse', 'eblc', 'data', 'eb']]
    arima_results = df[df.data == 'pweather']
    arima_results = get_tfe(arima_results)
    arima_results = arima_results[arima_results.eblc != 'baseline']
    arima_results.reset_index(drop=True, inplace=True)
    arima_results.eblc = arima_results.eblc.str.upper()
    arima_results['model'] = 'ARIMA'
    arima_results.data = arima_results.data.str.upper()
    arima_results['tfe_metric'] = 'nrmse'
    arima_results['te_metric'] = 'nrmse'
    arima_results['te'] = arima_results['error']
    x = arima_results.eb.values
    arima_results.eb = np.round(np.where(x >= 1.0, x*0.01, x), 3)
    arima_results = arima_results.rename({'nrmse': 'forecasting error', 'eblc': 'compression'}, axis=1)
    arima_results.drop('error', axis=1, inplace=True)
    pweather_results = pd.read_csv('../results/tfe/pprocessed_weather.csv')
    cr_df = pd.read_csv('../results/cr/pweather_cr.csv')
    cr_df.compression = cr_df.compression.str.upper()
    cr_df.data = cr_df.data.str.upper()
    cr_df.rename({'gzip': 'cr', 'error_bound': 'eb'}, axis=1, inplace=True)
    cr_df.set_index(['data', 'compression', 'eb'], inplace=True)
    joined = cr_df.merge(arima_results, on=['data', 'compression', 'eb'])
    concatenated = pd.concat([pweather_results, joined])
    concatenated['data'] = 'WEATHER'
    concatenated.to_csv('../results/tfe/pweather_results.csv', index=False)

In [80]:
join_arima_pweather()

D:\Anaconda\envs\dart\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [134]:
all_models_results = pd.read_csv('../results/tfe/all_models_results_.csv')
all_models_results = all_models_results[all_models_results.data != 'WEATHER']
all_models_results.drop('error', axis=1, inplace=True)
all_models_results.rename({'compression ratio': 'cr'}, axis=1, inplace=True)
pweather_results = pd.read_csv('../results/tfe/pweather_results.csv')
pd.concat([all_models_results, pweather_results]).to_csv('../results/tfe/models_results.csv', index=False)

In [130]:
all_models_results.head()

,compression,data,eb,tfe_metric,forecasting error,model,cr,TFE,te_metric,te
0,SZ,ETTM2,0.0,nrmse,0.048568,DLINEAR,1.0,0.0,nrmse,0.0
1,SZ,ETTM2,0.0,nrmse,0.048568,DLINEAR,1.0,0.0,rse,0.0
2,SZ,ETTM1,0.0,nrmse,0.064241,GRU,1.0,0.0,nrmse,0.0
3,SZ,ETTM1,0.0,nrmse,0.064241,GRU,1.0,0.0,rse,0.0
4,SZ,WIND,0.0,nrmse,0.032543,GRU,1.0,0.0,nrmse,0.0


In [131]:
pweather_results.head()

,eb,compression,tfe_metric,forecasting error,model,TFE,data,te_metric,te,cr
0,0.0,PMC,nrmse,0.069624,DLINEAR,0.0,WEATHER,nrmse,0.0,1.0
1,0.0,PMC,nrmse,0.069108,GRU,0.0,WEATHER,nrmse,0.0,1.0
2,0.0,PMC,nrmse,0.070381,INFORMER,0.0,WEATHER,nrmse,0.0,1.0
3,0.0,PMC,nrmse,0.073630,NBEATS,0.0,WEATHER,nrmse,0.0,1.0
4,0.0,PMC,nrmse,0.086052,TRANSFORMER,0.0,WEATHER,nrmse,0.0,1.0


,index,compression,data,eb,tfe_metric,forecasting error,model,cr,TFE,te_metric,te
0,0,SZ,ETTM2,0.00,nrmse,0.048568,DLINEAR,1.00,0.000000,nrmse,0.000000
1,1,SZ,ETTM2,0.00,nrmse,0.048568,DLINEAR,1.00,0.000000,rse,0.000000
2,2,SZ,ETTM1,0.00,nrmse,0.064241,GRU,1.00,0.000000,nrmse,0.000000
3,3,SZ,ETTM1,0.00,nrmse,0.064241,GRU,1.00,0.000000,rse,0.000000
4,4,SZ,WIND,0.00,nrmse,0.032543,GRU,1.00,0.000000,nrmse,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
4670,229,SWING,WEATHER,0.07,nrmse,0.142801,ARIMA,135.14,0.777471,nrmse,0.125848
4671,230,PMC,WEATHER,0.07,nrmse,0.119142,ARIMA,221.58,0.482981,nrmse,0.096297
4672,231,SWING,WEATHER,0.10,nrmse,0.186287,ARIMA,349.21,1.318749,nrmse,0.173962
4673,232,PMC,WEATHER,0.10,nrmse,0.148237,ARIMA,432.19,0.845136,nrmse,0.130697
